In [12]:
## Load CERES radiation anomaly dataset
ccf_anom_loRes = xr.open_dataset('/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/ccf_anom/ccf_anom_lo_res.nc')

SST_anom = ccf_anom_loRes.SST_anom
EIS_anom = ccf_anom_loRes.EIS_anom
Tadv_anom = ccf_anom_loRes.Tadv_anom
RH700_anom = ccf_anom_loRes.RH700_anom
w700_anom = ccf_anom_loRes.w700_anom
ws_anom = ccf_anom_loRes.ws_anom

In [ ]:
## Now, get low cloud cre anomaly for el nino/la nina years and split into seasons
def get_seas_comp(x):
    # Define empty numpy arrays to load data into
    count_en = range(0,len(el_nino_yrs))
    count_ln = range(0,len(la_nina_yrs))
    enso_JJA_back = []; enso_SON_back = []; enso_DJF = []; enso_MAM_for = []; enso_JJA_for = [];

    # For each el nino/la nina, get the mean for each season (JJA-1, SON-1, DJF, MAM+1, JJA+1) and append the empty arrays
    for i in count_en:
        en_start_year = el_nino_yrs[i][0]
        en_next_year  = el_nino_yrs[i][1]
        ln_start_year = la_nina_yrs[i][0]
        ln_next_year  = la_nina_yrs[i][1]

        JJA_back = (x.sel(time=slice(en_start_year+'-06',en_start_year+'-08')).mean(dim='time') -
                    x.sel(time=slice(ln_start_year+'-06',ln_start_year+'-08')).mean(dim='time'))

        SON_back = (x.sel(time=slice(en_start_year+'-09',en_start_year+'-11')).mean(dim='time') -
                    x.sel(time=slice(ln_start_year+'-09',ln_start_year+'-11')).mean(dim='time'))

        DJF      = (x.sel(time=slice(en_start_year+'-12',en_next_year+'-02')).mean(dim='time') -
                    x.sel(time=slice(ln_start_year+'-12',ln_next_year+'-02')).mean(dim='time'))

        MAM_for  = (x.sel(time=slice(en_next_year+'-03',en_next_year+'-05')).mean(dim='time') -
                    x.sel(time=slice(ln_next_year+'-03',ln_next_year+'-05')).mean(dim='time'))

        JJA_for  = (x.sel(time=slice(en_next_year+'-06',en_next_year+'-08')).mean(dim='time') -
                    x.sel(time=slice(ln_next_year+'-06',ln_next_year+'-08')).mean(dim='time'))

        enso_JJA_back.append(JJA_back)
        enso_SON_back.append(SON_back)
        enso_DJF.append(DJF)
        enso_MAM_for.append(MAM_for)
        enso_JJA_for.append(JJA_for)

    # Concatenate each numpy array with the seasonal mean for each el nino/la nina year, then average over all years
    JJA_back_enso_comp = xr.concat(enso_JJA_back,dim='count').assign_coords({'count':count_en}).mean(dim='count')
    SON_back_enso_comp = xr.concat(enso_SON_back,dim='count').assign_coords({'count':count_en}).mean(dim='count')
    DJF_enso_comp      = xr.concat(enso_DJF,dim='count').assign_coords({'count':count_en}).mean(dim='count')
    MAM_for_enso_comp  = xr.concat(enso_MAM_for,dim='count').assign_coords({'count':count_en}).mean(dim='count')
    JJA_for_enso_comp  = xr.concat(enso_JJA_for,dim='count').assign_coords({'count':count_en}).mean(dim='count')
    
    # Finally, put all DataArrays into one Dataset
    ccf_enso_composite = xr.Dataset(data_vars = {'JJA_back': JJA_back_enso_comp,
                                                   'SON_back': SON_back_enso_comp,
                                                   'DJF'     : DJF_enso_comp,
                                                   'MAM_for' : MAM_for_enso_comp,
                                                   'JJA_for' : JJA_for_enso_comp})
    
    # Get zonally averged CRE
    ccf_enso_composite_zonal = ccf_enso_composite.mean(dim='longitude').to_array() * np.cos(np.deg2rad(np.arange(-90,90,5)))
    ccf_enso_composite_zonal['variable'] = [0,1,2,3,4]
    
    # Get globally averged CRE
    ccf_enso_comp_avg = ccf_enso_composite.weighted(weight).mean(dim=('latitude','longitude')).round(3).to_array()
    ccf_enso_comp_avg['variable'] = [0,1,2,3,4]
    
    return ccf_enso_composite, ccf_enso_composite_zonal, ccf_enso_comp_avg

In [ ]:
SST_en_comp, SST_en_comp_zonal, SST_en_comp_glb = get_seas_comp(SST_anom)
EIS_en_comp, EIS_en_comp_zonal, EIS_en_comp_glb = get_seas_comp(EIS_anom)
Tadv_en_comp, Tadv_en_comp_zonal, Tadv_en_comp_glb = get_seas_comp(Tadv_anom)
RH700_en_comp, RH700_en_comp_zonal, RH700_en_comp_glb = get_seas_comp(RH700_anom)
w700_en_comp, w700_en_comp_zonal, w700_en_comp_glb = get_seas_comp(w700_anom)
ws_en_comp, ws_en_comp_zonal, ws_en_comp_glb = get_seas_comp(ws_anom)

In [ ]:
def get_enso_composites(seas_index):
    # Set up subplots figure with PlateCarree projection
    fig, ax = plt.subplots(1, 6, figsize=(24,3), subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180)})
    fig.tight_layout(w_pad=5)
    
    vmin = -1
    vmax = 1
    comp_title = ['SST', 'EIS', 'Tadv', 'RH700', '$\omega$700', 'Surf Wind Speed']
    seas_title = ['JJA-1', 'SON-1', 'DJF', 'MAM+1', 'JJA+1']
    
    # Plot Trends
    cbar1_data = SST_en_comp.get(seas_index).plot(ax=ax[0],transform=ccrs.PlateCarree(),cmap=plt.get_cmap('RdBu').reversed(),
                       add_colorbar=False,vmin=vmin,vmax=vmax)

    cbar2_data = EIS_en_comp.get(seas_index).plot(ax=ax[1],transform=ccrs.PlateCarree(),cmap=plt.get_cmap('RdBu').reversed(),
                       add_colorbar=False,vmin=vmin,vmax=vmax)

    cbar3_data = Tadv_en_comp.get(seas_index).plot(ax=ax[2],transform=ccrs.PlateCarree(),cmap=plt.get_cmap('RdBu').reversed(),
                       add_colorbar=False,vmin=vmin/2,vmax=vmax/2)

    cbar4_data = RH700_en_comp.get(seas_index).plot(ax=ax[3],transform=ccrs.PlateCarree(),cmap=plt.get_cmap('RdBu').reversed(),
                       add_colorbar=False,vmin=vmin*8,vmax=vmax*8)

    cbar5_data = w700_en_comp.get(seas_index).plot(ax=ax[4],transform=ccrs.PlateCarree(),cmap=plt.get_cmap('RdBu').reversed(),
                       add_colorbar=False,vmin=vmin*20,vmax=vmax*20)
    
    cbar6_data = ws_en_comp.get(seas_index).plot(ax=ax[5],transform=ccrs.PlateCarree(),cmap=plt.get_cmap('RdBu').reversed(),
                       add_colorbar=False,vmin=vmin,vmax=vmax)

     # Give each plot gridlines, coastlines, and block out continents
    for i in np.arange(0,6,1):
        ax[i].gridlines()
        ax[i].coastlines()
        ax[i].add_feature(cartopy.feature.LAND,zorder=100,facecolor='k',edgecolor='k')
        ax[i].set_aspect('auto')

        if seas_index == 'JJA_back':
            ax[i].set_title(comp_title[i], fontsize=20)
        elif seas_index == 'JJA_for':
            cbar_ax1 = fig.add_axes([0.018, -0.1, 0.13, 0.05])
            cbar1 = fig.colorbar(cbar1_data, cax=cbar_ax1, orientation='horizontal', extend='both')
            cbar1.ax.tick_params(labelsize=20)
            cbar1.set_label(label='$K$', size=20)

            cbar_ax2 = fig.add_axes([0.1855, -0.1, 0.13, 0.05])
            cbar2 = fig.colorbar(cbar2_data, cax=cbar_ax2, orientation='horizontal', extend='both')
            cbar2.ax.tick_params(labelsize=20)
            cbar2.set_label(label='$K$', size=20)

            cbar_ax3 = fig.add_axes([0.353, -0.1, 0.13, 0.05])
            cbar3 = fig.colorbar(cbar3_data, cax=cbar_ax3, orientation='horizontal', extend='both')
            cbar3.ax.tick_params(labelsize=20)
            cbar3.set_label(label='$K/day$', size=20)

            cbar_ax4 = fig.add_axes([0.519, -0.1, 0.13, 0.05])
            cbar4 = fig.colorbar(cbar4_data, cax=cbar_ax4, orientation='horizontal', extend='both')
            cbar4.ax.tick_params(labelsize=20)
            cbar4.set_label(label='%', size=20)

            cbar_ax5 = fig.add_axes([0.685, -0.1, 0.13, 0.05])
            cbar5 = fig.colorbar(cbar5_data, cax=cbar_ax5, orientation='horizontal', extend='both')
            cbar5.ax.tick_params(labelsize=20)
            cbar5.set_label(label='$hPa/day$', size=20)

            cbar_ax6 = fig.add_axes([0.852, -0.1, 0.13, 0.05])
            cbar6 = fig.colorbar(cbar6_data, cax=cbar_ax6, orientation='horizontal', extend='both')
            cbar6.ax.tick_params(labelsize=20)
            cbar6.set_label(label='$m/s$', size=20)
            ax[i].set_title(None)
        else:
            ax[i].set_title(None)

    fig.suptitle('Season = ' + seas_title[k], y=0.85, x=-0.01, rotation='vertical', fontsize=20)
    
    path = '/data/keeling/a/tjhanke2/enso-energy-budget/Output/Plots/seasonal_composites/'
#     fig.savefig(path + 'ccf_composites_' + seas_title[k] + '_seas.png', bbox_inches='tight')

In [ ]:
def get_zonal_enso_comp():
    # Set up subplots figure with PlateCarree projection
    fig, ax = plt.subplots(2, 3, figsize=(24,18))
    fig.tight_layout(w_pad=7,h_pad=10)
    
    vmin = -0.5
    vmax = 0.5
    comp_title = [['SST', 'EIS', 'Tadv'], 
                  ['RH700', '$\omega$700', 'Surf Wind Speed']]
    seas = ['JJA-1', 'SON-1', 'DJF', 'MAM+1', 'JJA+1']

    SST_en_comp_zonal.transpose().plot(ax=ax[0,0],cmap=plt.get_cmap('RdBu').reversed(),
                       cbar_kwargs={'shrink': 0.9,'label': '$K$'},vmin=vmin,vmax=vmax)
    
    EIS_en_comp_zonal.transpose().plot(ax=ax[0,1],cmap=plt.get_cmap('RdBu').reversed(),
                       cbar_kwargs={'shrink': 0.9,'label': '$K$'},vmin=vmin,vmax=vmax)
    
    Tadv_en_comp_zonal.transpose().plot(ax=ax[0,2],cmap=plt.get_cmap('RdBu').reversed(),
                       cbar_kwargs={'shrink': 0.9,'label': '$K/day$'},vmin=vmin,vmax=vmax)
    
    RH700_en_comp_zonal.transpose().plot(ax=ax[1,0],cmap=plt.get_cmap('RdBu').reversed(),
                       cbar_kwargs={'shrink': 0.9,'label': '%'},vmin=vmin*8,vmax=vmax*8)
    
    w700_en_comp_zonal.transpose().plot(ax=ax[1,1],cmap=plt.get_cmap('RdBu').reversed(),
                       cbar_kwargs={'shrink': 0.9,'label': '$hPa/day$'},vmin=vmin*20,vmax=vmax*20)
    
    ws_en_comp_zonal.transpose().plot(ax=ax[1,2],cmap=plt.get_cmap('RdBu').reversed(),
                       cbar_kwargs={'shrink': 0.9,'label': '$m/s$'},vmin=vmin,vmax=vmax)
    
    for i in range(0,2,1):
        for j in range(0,3,1):
            ax[i,j].set_xticks([0,1,2,3,4])
            ax[i,j].set_xticklabels(seas)
            ax[i,j].set_xlabel('Season')
            ax[i,j].set_title('Composited Anomalous Zonal Mean ' + comp_title[i][j] + ' Associated W/ ENSO vs Season')

    path = '/data/keeling/a/tjhanke2/enso-energy-budget/Output/Plots/seasonal_composites/'
#     fig.savefig(path + 'zonal_mean_ccf_composites_seas.png', bbox_inches='tight')

In [ ]:
def get_gm_ccf_comp():
    fig, ax = plt.subplots(figsize=(18,12))
    seas = ['JJA-1', 'SON-1', 'DJF', 'MAM+1', 'JJA+1']
    xaxis = [0,1,2,3,4]
    
    ax.plot(xaxis,SST_en_comp_glb, 'black', marker='o', label='SST')
    ax.plot(xaxis,EIS_en_comp_glb, 'red', marker='o', label='EIS')
    ax.plot(xaxis,Tadv_en_comp_glb, 'green', marker='o', label='Tadv')
    ax.plot(xaxis,RH700_en_comp_glb, 'blue', marker='o', label='RH700')
    ax.plot(xaxis,w700_en_comp_glb, 'purple', marker='o', label='$\omega$700')
    ax.plot(xaxis,ws_en_comp_glb, 'orange', marker='o', label='Surf WS')
    
    ax.set_xticks(xaxis)
    ax.set_xticklabels(seas)
    ax.set_yticks(np.linspace(-0.5,0.3,10))
    ax.axhline(y=0.0, color='k', linestyle='dotted')
    ax.axvline(x=2, color='k', linestyle='dotted')
    ax.set_xlabel('Season')
    #ax.set_ylabel('$W/m^2$')
    ax.legend()
        
    ax.set_title('Composited Anomalous Global Mean CCFs Associated w/ ENSO vs Season')
    
    path = '/data/keeling/a/tjhanke2/enso-energy-budget/Output/Plots/seasonal_composites/'
#     fig.savefig(path + 'glob_mean_ccf_composites_seas.png', bbox_inches='tight')